## 1. セットアップ

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 自作モジュールの読み込み
sys.path.append(os.path.abspath('..'))
from configs.config import *
from src.runner import Runner
from src.model_resnet_knn import ModelResNet50KNN
from src.util import Logger, Util, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 2. ロガーの設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    """run名の作成"""
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized (Local)")

## 3. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# ✅ リークしないCV戦略: quarter_sessionでグループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")

# ラベル分布
print(f"\nLabel distribution:")
print(df_train['label_id'].value_counts().sort_index())

In [ ]:
# ラベル分布の可視化
label_counts = df_train['label_id'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
label_counts.plot(kind='bar')
plt.title('Training Data Label Distribution')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 4. パラメータ設定（ローカル最適化）

In [ ]:
# メモ
memo = "ResNet50 + KNN method (Local)"

# run_name
run_name = get_run_name(model_type="resnet50_knn_local")
run_name = "resnet50_knn_local_202512141406"

print(f"Run name: {run_name}")

In [ ]:
# KNNモデルパラメータ（CPU/GPU自動判定）
import torch

# デバイス確認
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

params = {
    'model_name': 'resnet50',
    # KNN固有パラメータ
    'k': 5,
    # 閾値設定
    'threshold': 0.5,
    'min2_threshold': 0.3,
    # バッチサイズ（CPUなら小さめ、GPUなら大きめ）
    'batch_size': 64 if device == 'cuda' else 32,
    'num_workers': 4 if device == 'cuda' else 2,
    # キャッシュ設定
    'use_cache': True,
}

## 5. CV戦略設定

In [ ]:
# CV設定
cv_setting = {
    'method': 'group',  # 'group' (推奨) or 'stratified_group'
    'group_col': 'group',  # 上で作成したquarter_session列
    'n_splits': 5,
    # 注: GroupKFoldはshuffleパラメータなし（時系列順を保つ）
}

## 6. Runner作成

In [ ]:
# Runnerの作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50KNN,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger
)

print(f"\nRunner created: {run_name}")
print(f"Model class: {ModelResNet50KNN.__name__}")

In [ ]:
model = runner.build_model(i_fold=0)
model.load_model()

In [ ]:
test_features = model._extract_features_batch(df_test, split='test')

# 類似度計算（子クラスで実装）
similarities = model._compute_similarities(test_features)

In [ ]:
df_test

In [ ]:
DIR_IMAGE

In [ ]:
test_features

In [ ]:
set(similarities.sum(axis=1))

In [ ]:
from src import dataset_image
import importlib
importlib.reload(dataset_image)

In [ ]:
from src.dataset_image import BasketballDataset, get_transforms
from torch.utils.data import DataLoader

# BasketballDatasetの動作確認

# テスト用に小さいサンプルを取得
sample_df = df_train.head(20).copy()

# Dataset作成（学習用transform）
dataset_train = BasketballDataset(
    df=sample_df,
    transform=get_transforms(is_train=True),
    is_train=True,
)

# Dataset作成（評価用transform）
dataset_val = BasketballDataset(
    df=sample_df,
    transform=get_transforms(is_train=False),
    is_train=False,
)

print(f"✅ Dataset created successfully")
print(f"  Train dataset size: {len(dataset_train)}")
print(f"  Val dataset size: {len(dataset_val)}")

# 1サンプル取得して確認
image, label = dataset_train[0]

print(f"\n✅ Sample data:")
print(f"  Image shape: {image.shape}")
print(f"  Image dtype: {image.dtype}")
print(f"  Label: {label}")

# 画像の値の範囲確認（正規化されているか）
print(f"\n✅ Image statistics:")

print(f"  Min: {image.min():.3f}")
print(f"  Max: {image.max():.3f}")
print(f"  Mean: {image.mean():.3f}")
print(f"  Std: {image.std():.3f}")

# DataLoaderでバッチ処理が正常に動作するか確認

dataloader = DataLoader(
    dataset_train,
    batch_size=4,
    shuffle=False,
    num_workers=0  # 確認用に0に設定
)

batch_images, batch_labels = next(iter(dataloader))

print(f"\n✅ DataLoader batch:")
print(f"  Batch images shape: {batch_images.shape}")
print(f"  Batch labels shape: {batch_labels.shape}")

print(f"\n✅ All checks passed! BasketballDataset is working correctly.")

In [ ]:
next(iter(dataloader))

## 7. 5-Fold CV学習

In [ ]:
%%time
# 5-fold CV学習
# ⚠️ 初回実行時は特徴抽出に時間がかかります（CPU: 1-2時間, GPU: 20-30分）
# 2回目以降はキャッシュを使用するため高速です（数分）

scores = runner.train_cv()

print("\n" + "="*80)
print(f"CV Score (Macro F1): {np.mean(scores):.5f} ± {np.std(scores):.5f}")
print("="*80)

In [ ]:
# 2. 評価（OOF）
fold_scores, oof_score = runner.metric_cv()

print(f"OOF Score: {oof_score:.5f}")
print(f"Fold Scores: {fold_scores}")

## 8. テストデータ予測

In [ ]:
%%time
# CV全モデルのアンサンブル予測
pred_test = runner.predict_cv()

print(f"\nTest predictions shape: {pred_test.shape}")
print(f"\nPrediction distribution:")
print(pred_test['label_id'].value_counts().sort_index())

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (KNN - Local)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/11, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. 提出ファイル作成

In [ ]:
# 提出ファイル保存
submission_path = runner.save_submission(pred_test, run_name)

print(f"\n✅ Submission file saved: {submission_path}")
print(f"\nFirst 10 predictions:")
print(pred_test.head(10))

## 10. kのチューニング実験

In [ ]:
# kの値を変えて予測（類似度キャッシュ再利用で高速）
k_values = [3, 5, 7, 10, 15]
k_results = []

print("\n=== Testing different k values ===")
print("(Using cached similarities for fast tuning)\n")

for k in k_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=params['threshold'],
                min2_threshold=params['min2_threshold'],
                k=k
            )
            predictions_list.append(pred)
    
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        k_results.append({
            'k': k,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"k={k:2d}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_k_results = pd.DataFrame(k_results)
print("\n=== k-value tuning results ===")
print(df_k_results)

In [ ]:
# kとunknown比率の関係を可視化
if len(k_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_k_results['k'], df_k_results['unknown_ratio'], marker='o', linewidth=2)
    plt.xlabel('k (Number of neighbors)', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('KNN: Effect of k on Unknown Predictions (Local)', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

## 11. 閾値チューニング実験

In [ ]:
predictions_list

In [ ]:
# 閾値を変えて予測
threshold_values = np.arange(0.3, 0.8, 0.05)
threshold_results = []

print("\n=== Testing different threshold values ===")
print(f"(Using k={params['k']}, min2_threshold={params['min2_threshold']})\n")

for threshold in threshold_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=threshold,
                min2_threshold=params['min2_threshold'],
                k=params['k']
            )
            predictions_list.append(pred)
    
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        threshold_results.append({
            'threshold': threshold,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"threshold={threshold:.2f}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_threshold_results = pd.DataFrame(threshold_results)
print("\n=== Threshold tuning results ===")
print(df_threshold_results)

In [ ]:
# 閾値とunknown比率の関係を可視化
if len(threshold_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_threshold_results['threshold'], df_threshold_results['unknown_ratio'], 
             marker='o', linewidth=2, color='orange')
    plt.xlabel('Threshold', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('KNN: Effect of Threshold on Unknown Predictions (Local)', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

## 12. まとめ

### ✅ 実行完了内容
1. KNN法での5-fold CV学習（ローカル環境）
2. テストデータ予測
3. kのチューニング実験（k=3,5,7,10,15）
4. 閾値チューニング実験（threshold=0.3~0.8）
5. 提出ファイル作成

### 📊 KNN法の特徴
- ✅ 訓練データ全体（24,920サンプル）を活用
- ✅ kで柔軟に調整可能
- ⚠️ Prototype法より推論が遅い
- ⚠️ メモリ使用量が多い

### 🚀 次のステップ
- [ ] Prototype法と精度比較
- [ ] EfficientNet-B0で実験
- [ ] Prototype + KNNアンサンブル
- [ ] ArcFace導入検討

### 💾 結果の保存場所
- モデル: `models/`
- ログ: `logs/`
- 提出ファイル: `data/submission/`
- 特徴量キャッシュ: `data/features/`